In [9]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import sys
print(sys.executable)
sys.path.insert(1, '../src/')
from config import raw_data_path, univariate_data_path, processed_data_path, models_path
from preprocessing_modules import create_time_windows_with_labels, create_time_windows_with_metadata
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
import matplotlib.pyplot as plt
from FCMAE_model import FCMAE
from fastai.callback.tracker import SaveModelCallback, EarlyStoppingCallback


/home/nwertheim/miniconda3/bin/python


In [10]:
# Define FCMAE Hyperparameters:
batch_size = 16
num_blocks = 5
kernel_size = 7
base_dim = 32
learning_rate = 0.00016938000495408888
input_dimension = 512

num_epochs = 10

In [11]:

import torch
import numpy as np
import os
from torch.utils.data import TensorDataset, DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import optuna
from fastai.learner import Learner
from fastai.data.core import DataLoaders
from fastai.metrics import accuracy
from tsai.models.ResNet import ResNet
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from fastai.optimizer import SGD
# Load training data
train_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_train_2_80.npy")
train_data = np.load(train_dir, allow_pickle=True)
train_windows, train_labels, train_meta = create_time_windows_with_metadata(train_data)

# Convert to tensors
train_windows_tensor = torch.tensor(train_windows, dtype=torch.float32).unsqueeze(1)  # shape: (N, 1, 12000)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.long)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# Prepare dataset
full_dataset = TensorDataset(train_windows_tensor, train_labels_tensor)

In [12]:



# Load the pre-trained encoder
path = os.path.join(models_path, 'FCMAE_encoder_no_PCA_gpu_normalized_correct.pth')

# Load the full FCMAE model
fcmae = FCMAE(in_channels=1, base_dim=base_dim, num_blocks=num_blocks, kernel_size=kernel_size)


# Load the saved weights into the full model
state_dict = torch.load(path)
fcmae.load_state_dict(state_dict)

# Extract only the encoder from the FCMAE model
encoder = fcmae.encoder
# Set the encoder to evaluation mode and freeze the encoder parameters
encoder.eval()
for param in encoder.parameters():
    param.requires_grad = False  # Freeze the encoder

print(encoder)

Sequential(
  (0): Conv1d(1, 32, kernel_size=(7,), stride=(2,), padding=(3,))
  (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Conv1d(32, 64, kernel_size=(7,), stride=(2,), padding=(3,))
  (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Conv1d(64, 128, kernel_size=(7,), stride=(2,), padding=(3,))
  (7): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU()
  (9): Conv1d(128, 256, kernel_size=(7,), stride=(2,), padding=(3,))
  (10): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (11): ReLU()
  (12): Conv1d(256, 512, kernel_size=(7,), stride=(2,), padding=(3,))
  (13): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (14): ReLU()
)


In [13]:
'''tuneable optimizer, lower LR-range, early stopping'''
from fastai.learner import Metric
from sklearn.metrics import average_precision_score
from fastai.optimizer import SGD, Adam


# Custom Average Precision Metric
class AveragePrecision(Metric):
    def __init__(self):
        self.pred = []
        self.target = []

    def reset(self):
        self.pred, self.target = [], []

    def accumulate(self, learn):
        preds = learn.pred.detach().softmax(dim=-1)[:, 1]
        targs = learn.y.detach()
        self.pred.append(preds.cpu())
        self.target.append(targs.cpu())

    @property
    def value(self):
        preds = torch.cat(self.pred).numpy()
        targs = torch.cat(self.target).numpy()
        return average_precision_score(targs, preds)

    @property
    def name(self): return "avg_precision"

def objective(trial):
    lr = trial.suggest_loguniform('lr', 1e-7, 1e-6)
    optimizer_name = trial.suggest_categorical('optimizer', ['SGD', 'Adam'])

    # Map optimizer name to fastai optimizer function
    opt_func_map = {
    'SGD': SGD,
    'Adam': Adam,
    }

    opt_func = opt_func_map[optimizer_name]

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    avg_precisions = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(train_windows_tensor, train_labels_tensor)):
        train_subset = Subset(full_dataset, train_idx)
        val_subset = Subset(full_dataset, val_idx)

        train_loader = DataLoader(train_subset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=64, shuffle=False)
        dls = DataLoaders(train_loader, val_loader)

        class FCMAEClassifier(nn.Module):
            def __init__(self, encoder):
                super().__init__()
                self.encoder = encoder
                self.classifier = ResNet(input_dimension, 2)

            def forward(self, x):
                with torch.no_grad():
                    x = self.encoder(x)
                return self.classifier(x)

        model = FCMAEClassifier(encoder).to(device)
        loss_func = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

        learn = learn = Learner(
            dls,
            model,
            loss_func=loss_func,
            opt_func=opt_func,
            metrics=accuracy,
            cbs=[
                EarlyStoppingCallback(monitor='valid_loss', patience=3),
            ]
        )

        learn.fit_one_cycle(num_epochs, lr)
        ap = learn.validate()[1]  # [loss, avg_precision]
        avg_precisions.append(ap)

    return np.mean(avg_precisions)  # maximize this

# Run Optuna study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25)

print("Best trial:")
print(study.best_trial)


# Save the study
import joblib
joblib.dump(study, "FCMAE_ResNet_tune_80.pkl")


[I 2025-05-15 21:33:28,046] A new study created in memory with name: no-name-04ec4a0d-19a9-4fc8-8271-265743179d1e


epoch,train_loss,valid_loss,accuracy,time
0,0.706805,0.698517,0.463492,00:00
1,0.705398,0.704388,0.389206,00:00
2,0.707960,0.705494,0.447619,00:00
3,0.705986,0.700187,0.497143,00:00


No improvement since epoch 0: early stopping


epoch,train_loss,valid_loss,accuracy,time
0,0.700093,0.690621,0.554638,00:00
1,0.699178,0.694171,0.554638,00:00
2,0.696850,0.690925,0.554003,00:00
3,0.698964,0.690684,0.555273,00:00


No improvement since epoch 0: early stopping


...

epoch,train_loss,valid_loss,accuracy,time
0,0.695850,0.689627,0.648031,00:00
1,0.696402,0.690497,0.648666,00:00
2,0.696829,0.689287,0.627065,00:00
3,0.697022,0.690021,0.625159,00:00
4,0.696010,0.691039,0.638501,00:00
5,0.696133,0.690585,0.647395,00:00


No improvement since epoch 2: early stopping


[I 2025-05-15 21:45:00,353] Trial 24 finished with value: 0.4766999542713165 and parameters: {'lr': 1.9441393182774296e-07, 'optimizer': 'SGD'}. Best is trial 11 with value: 0.5662632942199707.


Best trial:
FrozenTrial(number=11, state=1, values=[0.5662632942199707], datetime_start=datetime.datetime(2025, 5, 15, 21, 39, 57, 31771), datetime_complete=datetime.datetime(2025, 5, 15, 21, 40, 17, 688659), params={'lr': 1.1568422321572289e-07, 'optimizer': 'SGD'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=1e-06, log=True, low=1e-07, step=None), 'optimizer': CategoricalDistribution(choices=('SGD', 'Adam'))}, trial_id=11, value=None)


['FCMAE_ResNet_tune_80.pkl']

In [14]:
print("Best trial:")
print(study.best_trial)


Best trial:
FrozenTrial(number=11, state=1, values=[0.5662632942199707], datetime_start=datetime.datetime(2025, 5, 15, 21, 39, 57, 31771), datetime_complete=datetime.datetime(2025, 5, 15, 21, 40, 17, 688659), params={'lr': 1.1568422321572289e-07, 'optimizer': 'SGD'}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=1e-06, log=True, low=1e-07, step=None), 'optimizer': CategoricalDistribution(choices=('SGD', 'Adam'))}, trial_id=11, value=None)


In [15]:
# '''MY DATA'''
# train_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_train2.npy")
# train_data = np.load(train_dir, allow_pickle=True)
# # train_data = pd.DataFrame(train_data)
# test_dir = os.path.join(univariate_data_path, "target_univariate_no_PCA_test2.npy")
# test_data = np.load(test_dir, allow_pickle=True)
# # test_data = pd.DataFrame(test_data)
# print(len(train_data))
# print(len(test_data))
# print(train_data[0])
# from torch.utils.data import DataLoader, TensorDataset, random_split
# from sklearn.utils.class_weight import compute_class_weight

# from collections import Counter

# # Example dataset with windows and labels
# train_windows, train_labels = create_time_windows_with_labels(train_data)
# test_windows, test_labels = create_time_windows_with_labels(test_data)

# # Count label distribution
# train_label_counts = Counter(train_labels)
# test_label_counts = Counter(test_labels)

# # Print the counts
# print("Train label distribution:")
# print(f"  Term (0): {train_label_counts[0]}")
# print(f"  Preterm (1): {train_label_counts[1]}")

# print("\nTest label distribution:")
# print(f"  Term (0): {test_label_counts[0]}")
# print(f"  Preterm (1): {test_label_counts[1]}")

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# # Convert to tensors
# train_windows_tensor = torch.tensor(train_windows, dtype=torch.float32)
# train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
# test_windows_tensor = torch.tensor(test_windows, dtype=torch.float32)
# test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

# train_windows_tensor = train_windows_tensor.unsqueeze(1)  # (1071, 1, 12000)
# test_windows_tensor = test_windows_tensor.unsqueeze(1)    # (899, 1, 12000)

# train_labels_tensor = train_labels_tensor.long()
# test_labels_tensor = test_labels_tensor.long()


# print(train_labels_tensor.shape)

# train_dataset = TensorDataset(train_windows_tensor, train_labels_tensor)
# test_dataset = TensorDataset(test_windows_tensor, test_labels_tensor)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [16]:
# '''CLASS WEIGHTS'''
# from tsai.models.ResNet import ResNet
# from tsai.models import ResNet
# from tsai.models.ResNet import ResNet
# from fastai.metrics import accuracy
# from fastai.data.core import DataLoaders
# from fastai.learner import Learner
# import torch

# class FCMAEClassifier(nn.Module):
#     def __init__(self, encoder):
#         super().__init__()
#         self.encoder = encoder
#         self.classifier = ResNet(2048, 2)  # 2048 channels from encoder, 2 output classes

#     def forward(self, x):
#         with torch.no_grad():  # freeze encoder
#             x = self.encoder(x)
#         return self.classifier(x)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Wrap your DataLoaders and set device
# dls = DataLoaders.from_dsets(
#     train_dataset,
#     test_dataset,
#     bs=32,
#     shuffle=True,
#     num_workers=0 
# )

# model = FCMAEClassifier(encoder)
# model.to(device)

# # Calculate class weights
# class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
# class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

# # Use weighted loss
# loss_func = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

# # Replace your loss function in the learner
# learn = Learner(dls, model, loss_func=loss_func, metrics=accuracy)
# learn.fit_one_cycle(20, 1e-6)

# learn.recorder.plot_loss()  # Plot loss curves (training and validation loss)
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.metrics import confusion_matrix
# import torch

# # Get predictions and true labels
# preds, targs = learn.get_preds(dl=learn.dls.valid)

# # Convert predictions to class labels (argmax for multi-class classification)
# pred_labels = preds.argmax(dim=1)

# # Compute confusion matrix
# cm = confusion_matrix(targs, pred_labels)

# # Plot confusion matrix
# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, 
#             xticklabels=['Class 0', 'Class 1'], yticklabels=['Class 0', 'Class 1'])
# plt.title('Confusion Matrix')
# plt.xlabel('Predicted Label')
# plt.ylabel('True Label')
# plt.show()

# from sklearn.metrics import (
#     accuracy_score,
#     precision_score,
#     recall_score,
#     f1_score,
#     average_precision_score,
#     roc_auc_score
# )

# # Convert to numpy arrays if needed
# true_labels = targs.cpu().numpy()
# pred_labels = pred_labels.cpu().numpy()
# pred_probs = preds[:, 1].cpu().numpy()  # Probability of class 1

# # Compute metrics
# accuracy = accuracy_score(true_labels, pred_labels)
# precision = precision_score(true_labels, pred_labels, zero_division=0)
# recall = recall_score(true_labels, pred_labels, zero_division=0)
# f1 = f1_score(true_labels, pred_labels, zero_division=0)
# ap = average_precision_score(true_labels, pred_probs)
# auc = roc_auc_score(true_labels, pred_probs)

# # Print results
# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")
# print(f"Average Precision (AP): {ap:.4f}")
# print(f"Area Under ROC Curve (AUC): {auc:.4f}")

